# Background

Here we present isotropic and anisotropic examples, both constrained to the same piecewise affine, continuous function.

The first is:
$$
X \sim (N(0, I_3) \; | \; \|X\|_1 = 1).
$$

The second is
$$
X \sim (N(0, D) \; | \; \|X\|_1 = 1)
$$
where $D$ is diagonal with diagonal elements $(10.0, 0.1, 0.1)$.

# Import

In [ ]:
import os
import numpy as np
import importlib as il

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Make sure this is set to the repository base directory
os.getenv("PYTHONPATH", "")

In [ ]:
il.import_module("ctgauss")

In [ ]:
from ctgauss import IsotropicCTGauss, AnisotropicCTGauss

# Define geometry of subspace

In [ ]:
F = np.eye(3)
F

In [ ]:
g = np.zeros((3,))
g

In [ ]:
region_signs = np.array([
    [1, 1, 1], # R1 = Q1, z > 0
    [-1, 1, 1], # R2 = Q2, z > 0
    [-1, -1, 1], # R3 = Q3, z > 0
    [1, -1, 1], # R4 = Q4, z > 0
    [1, 1, -1], # R5 = Q1, z < 0
    [-1, 1, -1], # R6 = Q2, z < 0
    [-1, -1, -1], # R7 = Q3, z < 0
    [1, -1, -1] # R8 = Q4, z < 0
])

In [ ]:
region_transitions = np.array([
    [2, 4, 5],
    [1, 3, 6],
    [4, 2, 7],
    [3, 1, 8],
    [6, 8, 1],
    [5, 7, 2],
    [8, 6, 3],
    [7, 5, 4],
])

In [ ]:
L = region_signs * region_transitions
L

In [ ]:
A = np.array([
    [1, 1, 1],
    [-1, 1, 1],
    [-1, -1, 1],
    [1, -1, 1],
    [1, 1, -1],
    [-1, 1, -1],
    [-1, -1, -1],
    [1, -1, -1],
]).reshape((8, 3, 1))

In [ ]:
y = -np.ones((8,1))

# Plot subspace

In [ ]:
# To plot |x|_1 cleanly we will create a mesh grid and then rotate it.

bound = 1. / np.sqrt(2)
xx, yy = np.meshgrid(np.linspace(-bound, bound, 101), np.linspace(-bound, bound, 101))

In [ ]:
alpha = np.pi / 4
R = np.matrix([
    [np.cos(alpha), -np.sin(alpha)],
    [np.sin(alpha), np.cos(alpha)]
])

In [ ]:
new_xy_flat = np.array([np.matmul(R, p) for p in zip(xx.flatten(), yy.flatten())]).squeeze()
new_xx = new_xy_flat[:,0].reshape(xx.shape)
new_yy = new_xy_flat[:,1].reshape(yy.shape)

In [ ]:
np.abs(new_xy_flat).sum(axis=1)

In [ ]:
z1 = 1 - (np.abs(new_xx) + np.abs(new_yy))
z2 = -1. * z1

In [ ]:
# Normal vectors, if we want to plot those
ntimes = 3
XYZ = np.zeros((8, 3, ntimes))
for i in range(8):
    XYZ[i,:,:] = np.outer(A[i,], np.linspace(0, 1.1, ntimes))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(new_xx, new_yy, z2, color="orange", alpha=0.5)
ax.plot_surface(new_xx, new_yy, z1, color="lightblue", alpha=0.5)
# for i in range(8):
#     ax.plot3D(XYZ[i,0,:], XYZ[i,1,:], XYZ[i,2,:], color="black", alpha=0.5)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title(r"The constrained region, $\|x\|_1 = 1$");

# Example - Isotropic Variance

In [ ]:
mu = np.zeros((3,))
phi = 1.0

N = 1000
t_max = 0.5*np.pi
x0 = np.array([1., 1., 1.]) / 3
x0dot = np.array([1., -1., 0.]) / 2
reg, j = 1, 0 # f vs. c indexing   

In [ ]:
rng = np.random.default_rng()

In [ ]:
ictg = IsotropicCTGauss(phi, mu, A, y, F, g, L)

In [ ]:
# DO WE PASS COTINUITY ERROR
ce = ictg.continuity_error()
np.all(ce < 1e15)

In [ ]:
(X, Xdot, R) = ictg.sample(rng, N, t_max, reg, x0, x0dot)
# (X, Xdot, R, I) = ictg.sample_with_boundaries(rng, N, t_max, reg, x0, x0dot)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(new_xx, new_yy, z2, color="orange", alpha=0.2)
ax.plot_surface(new_xx, new_yy, z1, color="lightblue", alpha=0.2)
ax.scatter3D(X[:,0], X[:,1], X[:,2], color="black", alpha=0.4, s=2)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title(r"$X \sim ( N(0, I_3) \; | \; \|X\|_1 = 1 )$");

In [ ]:
# Check that we are on |x|_1 = 1
np.allclose(np.abs(X).sum(axis=1), 1.0)

In [ ]:
# X.mean(axis=0)

# Example – Anisotropic Variance

In [ ]:
mu = np.zeros((3,))
Prec = np.array([
    [0.1, 0.0, 0.0],
    [0.0, 10.0, 0.0],
    [0.0, 0.0, 10.0]
])

N = 1000
t_max = 0.5*np.pi
x0 = np.array([1., 1., 1.]) / 3
x0dot = np.array([1., -1., 0.]) / 2
reg, j = 1, 0 # f vs. c indexing   

In [ ]:
rng = np.random.default_rng()

In [ ]:
actg = AnisotropicCTGauss(Prec, mu, A, y, F, g, L, mean=True)

In [ ]:
# DO WE PASS COTINUITY ERROR
ce = actg.continuity_error()
np.all(ce < 1e15)

In [ ]:
(XA, XAdot, RA) = actg.sample(rng, N, t_max, reg, x0, x0dot)
# (X, Xdot, R, I) = actg.sample_with_boundaries(rng, N, t_max, reg, x0, x0dot)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(new_xx, new_yy, z2, color="orange", alpha=0.2)
ax.plot_surface(new_xx, new_yy, z1, color="lightblue", alpha=0.2)
ax.scatter3D(XA[:,0], XA[:,1], XA[:,2], color="black", alpha=0.4, s=2)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title(r"$X \sim ( N(0, D) \; | \; \|X\|_1 = 1 )$, D=diag(0.1, 0.1, 10)");

In [ ]:
# Check that we are on |x|_1 = 1
np.allclose(np.abs(X).sum(axis=1), 1.0)

# Plot all together

In [ ]:
fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(1, 3, 1, projection='3d')
ax1.plot_surface(new_xx, new_yy, z2, color="orange", alpha=0.5)
ax1.plot_surface(new_xx, new_yy, z1, color="lightblue", alpha=0.5)
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_zlabel("z")
ax1.set_title(r"The constrained region, $\|x\|_1 = 1$");

ax2 = fig.add_subplot(1, 3, 2, projection='3d')
ax2.plot_surface(new_xx, new_yy, z2, color="orange", alpha=0.2)
ax2.plot_surface(new_xx, new_yy, z1, color="lightblue", alpha=0.2)
ax2.scatter3D(X[:,0], X[:,1], X[:,2], color="black", alpha=0.4, s=2)
ax2.set_xlabel("x")
ax2.set_ylabel("y")
ax2.set_zlabel("z")
ax2.set_title(r"$X \sim ( N(0, I_3) \; | \; \|X\|_1 = 1 )$");

ax3 = fig.add_subplot(1, 3, 3, projection='3d')
ax3.plot_surface(new_xx, new_yy, z2, color="orange", alpha=0.2)
ax3.plot_surface(new_xx, new_yy, z1, color="lightblue", alpha=0.2)
ax3.scatter3D(XA[:,0], XA[:,1], XA[:,2], color="black", alpha=0.4, s=2)
ax3.set_xlabel("x")
ax3.set_ylabel("y")
ax3.set_zlabel("z")
ax3.set_title(r"$X \sim ( N(0, D) \; | \; \|X\|_1 = 1 )$, D=diag(0.1, 10., 10.)");

# fig.savefig("onenorm-example.png", dpi=300, pad_inches=0)